## Ranking
#### Test

In [136]:
def precision_at_k(vector, k=None):
    
    if k is not None:
        if k > len(vector):
            raise ValueError("k cannot be more than length of vector")
        vector = vector[:k]
    
    return 1 / len(vector) * sum(vector)

def avg_precision_at_k(vector, k=None):
    
    if k is not None:
        if k > len(vector):
            raise ValueError("k cannot be more than length of vector")
        vector = vector[:k]

    numer = 0
    for i in range(len(vector)):
        numer += vector[i] * precision_at_k(vector[:i+1])

    return numer / sum(vector)

In [137]:
print(avg_precision_at_k([1,1,0,0]) - avg_precision_at_k([1,0,0,1]))

0.25


## Recommender system

In [173]:
import pandas as pd
import numpy as np
import itertools
from collections import OrderedDict

In [248]:
view = []
buy = []
with open('coursera_sessions_train.txt', 'r') as f:
    for line in f:
        l = line.split(';')
        v = (l[0].split(','))
        b = (l[1].split(','))
        if v[0] == '\n':
            continue
        if b[0] != '\n':
            temp = b[-1].split('\n')[0]
            b[-1] = temp
            buy.append(b)
        
        view.append(v)

In [249]:
view = np.array(list(itertools.chain(*view))).astype(int)
buy = np.array(list(itertools.chain(*buy))).astype(int)

In [250]:
len(view)

356177

In [257]:
d_view, d_buy = {}, {}

for i in view:
    d_view.setdefault(i, 0)
    d_view[i] += 1
    
for i in buy:
    d_buy.setdefault(i, 0)
    d_buy[i] += 1

In [252]:
#for k, v in d_view.items():
#    d_view[k] = d_view[k] / len(view)
    
#for k, v in d_buy.items():
#    d_buy[k] = d_buy[k] / len(buy)

In [258]:
#2
popularity_train = sorted(d_view.items(), key=lambda x: x[1], reverse=True)
purchase_train = sorted(d_buy.items(), key=lambda x: x[1], reverse=True)

In [239]:
with open('coursera_sessions_test.txt', 'r') as f:
    test = f.readlines()

In [240]:

# Reading train and test data
with open('coursera_sessions_train.txt', 'r') as f:
    sess_train = f.read().splitlines()
with open('coursera_sessions_test.txt', 'r') as f:
    sess_test = f.read().splitlines()

In [241]:

# Create train array splitted by looks (look_items) and purchases (pur_items)
sess_train_lp = []
for sess in sess_train:
    look_items, pur_items = sess.split(';')
    look_items = list(map(int, look_items.split(',')))
    if len(pur_items) > 0:
        pur_items = list(map(int, pur_items.split(',')))
    else:
        pur_items = []
    sess_train_lp.append([look_items, pur_items])

#test
sess_test_lp = []
for sess in sess_test:
    look_items, pur_items = sess.split(';')
    look_items = map(int, look_items.split(','))
    if len(pur_items) > 0:
        pur_items = map(int, pur_items.split(','))
    else:
        pur_items = []
    sess_test_lp.append([look_items, pur_items])

In [242]:
sess_train_l = [row[0] for row in sess_train_lp]
sess_train_l_np = np.array( [id_n for sess in sess_train_l for id_n in sess] )

# Array of unique ids and looks in train data
sess_train_l_cnt = np.transpose(np.unique(sess_train_l_np, return_counts=True)) #==d_view

In [243]:
#test
# Array of purchases
sess_train_p = [row[1] for row in sess_train_lp]
sess_train_p_np = np.array( [id_n for sess in sess_train_p for id_n in sess] )

# Array of unique ids and purchases in train dataset
sess_train_p_cnt = np.transpose(np.unique(sess_train_p_np, return_counts=True))

In [244]:
len(sess_train_l_np), len(sess_train_p_np), len(sess_train_l_cnt), len(sess_train_p_cnt)

(356177, 5374, 77064, 4479)

In [245]:

# Sorting arrays of looks and purchases by counts
sess_train_l_cnt = sess_train_l_cnt[sess_train_l_cnt[:,1].argsort()][::-1]
sess_train_p_cnt = sess_train_p_cnt[sess_train_p_cnt[:,1].argsort()][::-1]

In [246]:
def prec_rec_metrics(session, reccomendations, k):
    purchase = 0
    for ind in reccomendations:
        if ind in session:
            purchase += 1 
    precision = purchase / k
    recall = purchase / len(session)
    return(precision, recall)

In [216]:
prec_at_1_tr_l, rec_at_1_tr_l = [], []
prec_at_5_tr_l, rec_at_5_tr_l = [], []
k1, k5 = 1, 5
for i, sess_p in enumerate(sess_train_p):
    # skip sessions without purchases
    if sess_p == []: continue
    
    # looks ids
    sess_l = sess_train_l[i]

    # sorted looks ids indices in sess_train_l_cnt array
    # sort in accordance with looks counts
    l_ind_sess = []
    for j in range(len(sess_l)):
        l_ind_sess.append(np.where(sess_train_l_cnt[:,0] == sess_l[j])[0][0])
    l_ind_sess_sorted = np.unique(l_ind_sess)
    
    # k1 recommendations
    num_of_recs_k1 = min(k1, len(sess_l))
    if num_of_recs_k1 == 0: continue
    recs_k1 = sess_train_l_cnt[l_ind_sess_sorted[:num_of_recs_k1],0]
    
    # k1 metrics
    prec_1, rec_1 = prec_rec_metrics(sess_p, recs_k1, k1)
    prec_at_1_tr_l.append(prec_1)
    rec_at_1_tr_l.append(rec_1)
    
    # k5 recommendations
    num_of_recs_k5 = min(k5, len(sess_l))
    if num_of_recs_k5 == 0: continue
    recs_k5 = sess_train_l_cnt[l_ind_sess_sorted[:num_of_recs_k5],0]
    
    # k5 metrics
    prec_5, rec_5 = prec_rec_metrics(sess_p, recs_k5, k5)
    prec_at_5_tr_l.append(prec_5)
    rec_at_5_tr_l.append(rec_5)

In [217]:
avg_prec_at_1_tr_l = np.mean(prec_at_1_tr_l)
avg_rec_at_1_tr_l = np.mean(rec_at_1_tr_l)
avg_prec_at_5_tr_l = np.mean(prec_at_5_tr_l)
avg_rec_at_5_tr_l = np.mean(rec_at_5_tr_l)

In [218]:
r1 = '%.2f' % round(avg_rec_at_1_tr_l, 2)
p1 = '%.2f' % round(avg_prec_at_1_tr_l, 2)
r5 = '%.2f' % round(avg_rec_at_5_tr_l, 2)
p5 = '%.2f' % round(avg_prec_at_5_tr_l, 2)
ans1 = ' '.join([r1, p1, r5, p5])
print('Answer 1:', ans1)

Answer 1: 0.44 0.51 0.83 0.21
